## Parsing the CMap
In this part, the cmapBQ API package will be used to parse the CMap and query the genetic perturbation signatures induced in CNS cell lines and their relevant metadata tables. We will first parse the metadata table of genetic perturbations to collect the IDs of all signatures of this genetic type. Then we will parse the cell information table to select only cell lines from central nervous system lineages. Finally, we will import the disease set of genes (disease signature) to formulate the final query based on the specific signature IDs and the disease gene set we are interested in.

In [1]:
import os
import pandas as pd
import numpy as np
import requests

In [2]:
import cmapBQ.query as cmap_query
import cmapBQ.config as cmap_config

In [3]:
# Get autherization 
# The crediatals file is not uploaded
cmap_config.setup_credentials("./connectivitymap-1ec673279e60.json")
from google.cloud import bigquery
credentials_filepath = "./connectivitymap-1ec673279e60.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_filepath
bq_client = bigquery.Client()

### List the CMap tables
These are the cmap-big-tables that include different levels of data as well as metadata for the experiments. Our target is the table containing level-5 data which are the final signatures.

In [4]:
import cmapBQ.query as cmap_query

cmap_query.list_tables()

_includes_clustered_tables: <bound method TableDirectory._includes_clustered_tables of TableDirectory(compoundinfo='cmap-big-table.cmap_lincs_public_views.compoundinfo', genetic_pertinfo='cmap-big-table.cmap_lincs_public_views.genetic_pertinfo', geneinfo='cmap-big-table.cmap_lincs_public_views.geneinfo', cellinfo='cmap-big-table.cmap_lincs_public_views.cellinfo', instinfo='cmap-big-table.cmap_lincs_public_views.instinfo', siginfo='cmap-big-table.cmap_lincs_public_views.siginfo', level3='cmap-big-table.cmap_lincs_public_views.L1000_Level3_cid', level3_rid='cmap-big-table.cmap_lincs_public_views.L1000_Level3_rid', level3_landmark='cmap-big-table.cmap_lincs_public_views.L1000_Level3_landmark', level4='cmap-big-table.cmap_lincs_public_views.L1000_Level4_cid', level4_rid='cmap-big-table.cmap_lincs_public_views.L1000_Level4_rid', level4_landmark='cmap-big-table.cmap_lincs_public_views.L1000_Level4_landmark', level5='cmap-big-table.cmap_lincs_public_views.L1000_Level5_cid', level5_rid='cmap-b

### Metadata for Level-5 data

In [ ]:
query = "SELECT COUNT(DISTINCT(cid)) as num_level5_sigs FROM cmap-big-table.cmap_lincs_public_views.L1000_Level5"

cmap_query.run_query(query=query, client=bq_client).result().to_dataframe()

In [5]:
sig_metadata = cmap_query.cmap_sig(bq_client)
sig_metadata.head()

,sig_id,pert_id,cmap_name,pert_type,cell_iname,pert_itime,pert_idose,nsample,det_plates,build_name,project_code,ss_ngene,cc_q75,tas
0,ABY001_NCIH508_XH:BEVACIZUMAB:0.5:24,BEVACIZUMAB,BEVACIZUMAB,trt_aby,NCIH508,24 h,0.5 mg/ml,3,ABY001_NCIH508_XH_X1_B15,None,ABY,189,0.370000,0.267401
1,ABY001_NCIH2073_XH:BEVACIZUMAB:0.0625:24,BEVACIZUMAB,BEVACIZUMAB,trt_aby,NCIH2073,24 h,0.06 mg/ml,3,ABY001_NCIH2073_XH_X1_B15,None,ABY,121,0.090000,0.105522
2,AML001_CD34_6H:BRD-K68552125:10,BRD-K68552125,BRD-K68552125,trt_cp,CD34,6 h,10 uM,3,AML001_CD34_6H_X1_F1B10|AML001_CD34_6H_X2_F1B1...,None,AML,569,0.820000,0.690707
3,CNS001_HA1E_96H:EMPTY_VECTOR:-666,CNS001-EMPTY_VECTOR,EMPTY_VECTOR,ctl_vector.cns,HA1E,96 h,None,41,None,None,CNS,681,0.763151,0.728970
4,CPD003_PC3_6H:D03,BRD-K54094468,remoxipride,trt_cp,PC3,6 h,10 uM,1,CPD003_PC3_6H_X3.A2_B6_DUO52HI53LO,None,CPD,0,0.000000,0.000000


In [6]:
# Truncation of metadata table 
sig_metadata = sig_metadata.iloc[:,0:7]

In [7]:
sig_metadata.head()

,sig_id,pert_id,cmap_name,pert_type,cell_iname,pert_itime,pert_idose
0,ABY001_NCIH508_XH:BEVACIZUMAB:0.5:24,BEVACIZUMAB,BEVACIZUMAB,trt_aby,NCIH508,24 h,0.5 mg/ml
1,ABY001_NCIH2073_XH:BEVACIZUMAB:0.0625:24,BEVACIZUMAB,BEVACIZUMAB,trt_aby,NCIH2073,24 h,0.06 mg/ml
2,AML001_CD34_6H:BRD-K68552125:10,BRD-K68552125,BRD-K68552125,trt_cp,CD34,6 h,10 uM
3,CNS001_HA1E_96H:EMPTY_VECTOR:-666,CNS001-EMPTY_VECTOR,EMPTY_VECTOR,ctl_vector.cns,HA1E,96 h,None
4,CPD003_PC3_6H:D03,BRD-K54094468,remoxipride,trt_cp,PC3,6 h,10 uM


### Metadata for genetic pertutbations

In [8]:
genpert_info = cmap_query.cmap_genetic_perts(client=bq_client)

# Show a sample
genpert_info.sample(10)

,pert_id,cmap_name,pert_type,gene_id,gene_title,ensembl_id,gene_type,feature_space
41269,BRDN0001485059,NDUFS1,trt_xpr,4719,NADH:ubiquinone oxidoreductase core subunit S1,ENSG00000023228,protein-coding,best inferred
22084,BRDN0001149366,RIOK2,trt_xpr,55781,RIO kinase 2,ENSG00000058729,protein-coding,best inferred
33568,TRCN0000022184,ARHGAP35,trt_sh,2909,Rho GTPase activating protein 35,ENSG00000160007,protein-coding,best inferred
40179,TRCN0000036495,ARPC1B,trt_sh,10095,actin related protein 2/3 complex subunit 1B,ENSG00000130429,protein-coding,best inferred
35795,BRDN0001518931,TTC37,trt_xpr,9652,tetratricopeptide repeat domain 37,ENSG00000198677,protein-coding,best inferred
40824,TRCN0000468230,CYP3A5,trt_oe,1577,cytochrome P450 family 3 subfamily A member 5,ENSG00000106258,protein-coding,best inferred
4871,BRDN0001497958,VPREB1,trt_xpr,7441,V-set pre-B cell surrogate light chain 1,ENSG00000169575,protein-coding,inferred
31880,BRDN0001495292,IL1R1,trt_xpr,3554,interleukin 1 receptor type 1,ENSG00000115594,protein-coding,best inferred
41590,CGS001-5295,PIK3R1,trt_sh.cgs,5295,phosphoinositide-3-kinase regulatory subunit 1,ENSG00000145675,protein-coding,best inferred
7005,TRCN0000039854,CHEK1,trt_sh,1111,checkpoint kinase 1,ENSG00000149554,protein-coding,landmark


In [9]:
genpert_info.shape

(44769, 8)

### Types of genetic perturbations
These are the genetic modifications used to induce the signatures. These will be used as edge types in the heterogeneous graph where each perturbation node will be connected to its target via an edge of a type of one of these genetic modifications.

In [10]:
genpert_info['pert_type'].unique()

array(['trt_oe', 'trt_sh', 'trt_sh.cgs', 'trt_si', 'trt_xpr'],
      dtype=object)

### Cell lines from the *cell-info* table 
To collect distinct *cell_inames* of the Central Nervous System (CNS) cell lineages.

In [11]:
qu = "SELECT DISTINCT cell_iname FROM cmap-big-table.cmap_lincs_public_views.cellinfo WHERE cell_lineage = 'central_nervous_system'"
cmap_query.run_query(query=qu, client=bq_client).result().to_dataframe()

,cell_iname
0,NEU
1,NPC
2,LN229
3,SKNSH
4,U251MG
5,YH13
6,GI1


In [12]:
## All genetic signatures of CNS cells
query = "SELECT DISTINCT(sig_id) FROM cmap-big-table.cmap_lincs_public_views.siginfo WHERE cell_iname IN ('NEU','NPC','LN229','SKNSH','U251MG','YH13','GI1') AND pert_type IN ('trt_oe', 'trt_sh', 'trt_sh.cgs', 'trt_si', 'trt_xpr')"

genetic_sig_ids = cmap_query.run_query(query=query, client=bq_client).result().to_dataframe()


In [13]:
genetic_cns_sig_meta = sig_metadata[sig_metadata['sig_id'].isin(genetic_sig_ids['sig_id'])]
genetic_cns_sig_meta.head()

,sig_id,pert_id,cmap_name,pert_type,cell_iname,pert_itime,pert_idose
4527,TSAI001_221NPC1_XH:HDAC1-SHRNA2:-666,HDAC1-SHRNA2,HDAC1-SHRNA2,trt_xpr,NPC,None,2 uM
42630,TSAI001_APP11NPC1_XH:HDAC1-SHRNA4:-666,HDAC1-SHRNA4,HDAC1-SHRNA4,trt_xpr,NPC,None,2 uM
43722,TSAI001_APP11NPC1_XH:HDAC1-SHRNA5:-666,HDAC1-SHRNA5,HDAC1-SHRNA5,trt_xpr,NPC,None,2 uM
45733,TSAI001_221NPC1_XH:HDAC1-SHRNA4:-666,HDAC1-SHRNA4,HDAC1-SHRNA4,trt_xpr,NPC,None,2 uM
73873,TSAI001_221NPC1_XH:HDAC1-SHRNA5:-666,HDAC1-SHRNA5,HDAC1-SHRNA5,trt_xpr,NPC,None,2 uM


In [14]:
genetic_cns_sig_meta.shape

(20635, 7)

In [15]:
genperts_cns_metadata = genpert_info[genpert_info['pert_id'].isin(genetic_cns_sig_meta['pert_id'])]
genperts_cns_metadata = genperts_cns_metadata.dropna()
genperts_cns_metadata.head()

,pert_id,cmap_name,pert_type,gene_id,gene_title,ensembl_id,gene_type,feature_space
2467,BRDN0001487806,REN,trt_xpr,5972,renin,ENSG00000143839,protein-coding,inferred
2468,BRDN0001483163,REN,trt_xpr,5972,renin,ENSG00000143839,protein-coding,inferred
2469,BRDN0001501866,TBX4,trt_xpr,9496,T-box 4,ENSG00000121075,protein-coding,inferred
2470,BRDN0001502500,TBX4,trt_xpr,9496,T-box 4,ENSG00000121075,protein-coding,inferred
2475,BRDN0001488775,ALB,trt_xpr,213,albumin,ENSG00000163631,protein-coding,inferred


In [17]:
# Map signature ids in the metadata table to perturbation ids in genetic_perturbations table
merged_df = pd.merge(genperts_cns_metadata, genetic_cns_sig_meta[['sig_id', 'pert_id']], on='pert_id', how='left')
merged_df = merged_df.astype('str')
merged_df.head()

,pert_id,cmap_name,pert_type,gene_id,gene_title,ensembl_id,gene_type,feature_space,sig_id
0,BRDN0001487806,REN,trt_xpr,5972,renin,ENSG00000143839,protein-coding,inferred,XPR009_U251MG.311_96H:D02
1,BRDN0001483163,REN,trt_xpr,5972,renin,ENSG00000143839,protein-coding,inferred,XPR009_U251MG.311_96H:H22
2,BRDN0001501866,TBX4,trt_xpr,9496,T-box 4,ENSG00000121075,protein-coding,inferred,XPR025_U251MG.311_96H:J07
3,BRDN0001502500,TBX4,trt_xpr,9496,T-box 4,ENSG00000121075,protein-coding,inferred,XPR025_U251MG.311_96H:N11
4,BRDN0001488775,ALB,trt_xpr,213,albumin,ENSG00000163631,protein-coding,inferred,XPR009_U251MG.311_96H:P11


In [18]:
# Use gene_mapping_dictionary to select only signatures with targets present in the hetero-graph
import json
file_path = '/home/ahmed_kh/project/graph/genes/edge_index/mapping_dicts/genes_dict.json'
with open(file_path, 'r') as json_file:
    genes_dict_json = json.load(json_file)

#convert back strings into integers
    genes_dict = {int(key): value for key, value in genes_dict_json.items()}
    
print(len(genes_dict))

18315


In [19]:
genes = list({str(key) for key, value in genes_dict.items()})
merged_df = merged_df[merged_df['gene_id'].isin(genes)]
merged_df.head()

,pert_id,cmap_name,pert_type,gene_id,gene_title,ensembl_id,gene_type,feature_space,sig_id
0,BRDN0001487806,REN,trt_xpr,5972,renin,ENSG00000143839,protein-coding,inferred,XPR009_U251MG.311_96H:D02
1,BRDN0001483163,REN,trt_xpr,5972,renin,ENSG00000143839,protein-coding,inferred,XPR009_U251MG.311_96H:H22
2,BRDN0001501866,TBX4,trt_xpr,9496,T-box 4,ENSG00000121075,protein-coding,inferred,XPR025_U251MG.311_96H:J07
3,BRDN0001502500,TBX4,trt_xpr,9496,T-box 4,ENSG00000121075,protein-coding,inferred,XPR025_U251MG.311_96H:N11
4,BRDN0001488775,ALB,trt_xpr,213,albumin,ENSG00000163631,protein-coding,inferred,XPR009_U251MG.311_96H:P11


This is now the final list of signature IDs that we want to download.

### *Perturbation::target* relathionships
This is the table that will be used to create the "perturbation::target" relationship module in the graph

In [20]:
signature_moa_target = merged_df[['sig_id','pert_type','gene_id']].copy()
signature_moa_target.head()

,sig_id,pert_type,gene_id
0,XPR009_U251MG.311_96H:D02,trt_xpr,5972
1,XPR009_U251MG.311_96H:H22,trt_xpr,5972
2,XPR025_U251MG.311_96H:J07,trt_xpr,9496
3,XPR025_U251MG.311_96H:N11,trt_xpr,9496
4,XPR009_U251MG.311_96H:P11,trt_xpr,213


In [21]:
signature_moa_target.shape

(20023, 3)

In [22]:
# Save metadata as a csv file
csv_filename = 'signature_moa_target.csv'
signature_moa_target.to_csv(csv_filename, index=False)

### Signature querying

In [5]:
# signatures to be queried
signatures = signature_moa_target['sig_id'].to_list()

# Gene IDs to be queried
AD_signature = pd.read_csv("AD_signature.csv")
AD_signature = AD_signature.astype('str')
AD_signature = AD_signature['Gene.ID'].to_list()
AD_sig = AD_sig.iloc[:,1:-1]

In [32]:
# Set the batch size
batch_size = 1000
df = pd.DataFrame()

# Loop through the list in batches
for i in range(0, len(signatures), batch_size):
    batch = signatures[i:i+batch_size]
   
    # Run the query
    data_numerical = cmap_query.cmap_matrix(bq_client, data_level='level5', feature_space='bing', cid=batch, rid=AD_signature, chunk_size=1000)

    # Convert the GCTOO matrix to a pandas DataFrame
    dfx = data_numerical.data_df
    
    dft = dfx.transpose()
    
    df = pd.concat([df,dft],axis=0)
    
print("Done!")

Running query ... (1/1)
Total bytes processed: 9.7GiB
Total bytes billed: 9.7GiB
Pivoting Dataframes to GCT objects
Complete
Running query ... (1/1)
Total bytes processed: 14.1GiB
Total bytes billed: 14.1GiB
Pivoting Dataframes to GCT objects
Complete
Running query ... (1/1)
Total bytes processed: 13.1GiB
Total bytes billed: 13.1GiB
Pivoting Dataframes to GCT objects
Complete
Running query ... (1/1)
Total bytes processed: 11.9GiB
Total bytes billed: 11.9GiB
Pivoting Dataframes to GCT objects
Complete
Running query ... (1/1)
Total bytes processed: 13.3GiB
Total bytes billed: 13.3GiB
Pivoting Dataframes to GCT objects
Complete
Running query ... (1/1)
Total bytes processed: 12.2GiB
Total bytes billed: 12.2GiB
Pivoting Dataframes to GCT objects
Complete
Running query ... (1/1)
Total bytes processed: 13.2GiB
Total bytes billed: 13.2GiB
Pivoting Dataframes to GCT objects
Complete
Running query ... (1/1)
Total bytes processed: 12.1GiB
Total bytes billed: 12.1GiB
Pivoting Dataframes to GCT obj

In [33]:
df.shape

(20023, 6056)

In [34]:
df.iloc[:10,:10]

rid,100,10000,10001,10005,10006,10007,10010,100129250,100129482,100131755
cid,,,,,,,,,,
CGS001_NPC_96H:HBE1:1.5,-0.218373,0.008493,-0.480829,0.166468,-0.328992,-0.121826,0.136187,-0.508528,0.243526,0.635878
CGS001_NPC_96H:LHX3:1.5,-0.570863,-0.502525,0.165043,0.244646,0.117853,0.229546,0.008000,0.414267,-0.613329,-0.282516
CGS001_NPC_96H:PRKCE:1.5,0.347789,0.293864,-0.517908,-0.106490,0.486904,-0.141105,0.551190,0.077079,0.066313,0.667602
CGS001_NPC_96H:SENP5:1.5,0.376575,0.393675,-0.609250,0.546325,0.045675,-0.718825,0.562125,0.346800,0.215425,0.182775
KDB005_NPC_96H:TRCN0000062310:-666,-0.058650,-0.945650,0.435700,0.466750,0.124650,-1.131450,0.509300,0.741000,-0.466700,-0.551050
KDB005_NPC_96H:TRCN0000062311:-666,0.811800,1.733000,-1.654200,0.625900,-0.033300,-0.306200,0.614950,-0.047400,0.897550,0.916600
KDB006_NPC_96H:K06,-0.212064,-0.372232,-0.055138,0.247922,-0.265903,0.701970,0.210728,-0.730278,-0.457543,0.144343
KDB006_NPC_96H:K14,0.247382,0.426326,-1.069436,-0.204103,0.434191,-1.240098,0.259844,-0.158531,0.750229,1.785550
KDB006_NPC_96H:M18,-1.155408,0.335541,-0.147187,0.549799,-0.597942,0.785241,-0.809739,-0.270088,0.826635,0.246780


In [35]:
# Mapping IDs and sorting the dataframe
file_path = 'mapping_dicts/genperts_dict.json'
with open(file_path, 'r') as json_file:
    genperts_json = json.load(json_file)

    genperts_dict = {str(key): value for key, value in genperts_json.items()}
    
print(len(genperts_dict))

20023


In [3]:
df.index = df.index.map(genperts_dict)
df = df.sort_index()
df

,100,10000,10001,10005,10006,10007,10010,100129250,100129482,100131755,...,9973,9976,9978,998,9980,9987,9988,9990,9991,9997
0,0.174898,0.170660,-0.327706,-1.012201,-0.709069,0.590302,0.317491,-0.336832,-0.622616,-1.192824,...,-0.020990,-0.943036,-0.457074,-0.451099,-0.275676,0.025439,-0.940749,0.108191,0.012258,0.436765
1,0.023692,-0.621700,-0.214531,0.067540,-0.066525,0.305235,0.673813,0.026184,-0.026370,0.469255,...,0.625724,0.280934,-0.312827,0.258809,0.337565,-0.288991,-0.516484,0.399671,-0.623557,-0.335016
2,0.406850,0.775148,-0.288365,1.054073,-0.470195,0.224927,-0.055932,-0.041696,-0.939581,0.170502,...,-0.822419,0.436913,-0.108121,0.189611,0.088665,-0.411371,0.159655,-0.312251,-0.004310,0.327478
3,0.012266,0.370053,0.905270,0.524906,1.090123,-1.512671,-0.746616,-0.560953,-0.119623,-0.714175,...,-0.903741,-0.134229,0.230682,0.673075,-0.450610,0.643537,-0.437256,-0.244052,-0.270511,-0.391445
4,0.680962,-1.441959,0.123246,-0.414082,-0.119506,-0.310105,0.768365,0.300217,0.037321,0.057293,...,-1.274699,0.608310,-0.498205,-1.115365,-0.178106,-0.262965,0.487434,-0.042183,1.015008,-0.002290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20018,1.630630,-1.057249,-1.483222,-0.694036,0.392080,-0.302878,-0.181074,-1.345645,-0.120782,1.596910,...,0.896386,0.376598,0.010688,-0.353705,-0.522536,-0.718549,-1.132831,-0.174150,-0.610226,0.576494
20019,1.910709,-0.613249,-1.384393,1.606892,-0.516433,-0.464828,0.489844,0.257911,-0.760184,0.645760,...,1.832853,-0.387472,-0.745996,-0.018421,-0.794105,-0.904071,-0.857446,-0.791354,1.414317,-0.423272
20020,1.285126,0.607435,-1.041552,-0.743960,-0.831314,-0.496065,-0.702506,0.159211,0.202441,1.274807,...,1.931300,-0.839873,0.334764,-0.183769,-0.945705,-0.867503,-0.121041,0.739108,-0.742083,0.058644
20021,0.069787,-0.865963,-0.208207,-0.925451,-0.413606,0.946464,0.211409,-0.306070,-1.471482,-1.303913,...,-0.597694,-0.450268,-0.335929,0.760301,-0.961826,0.312094,-0.615488,-1.047882,0.030394,-0.519334


In [42]:
# Save the signatures as a csv file
df.to_csv("20023_6056_signatures.csv", index=True)